In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from os.path import isdir, join
from os import mkdir

_default_date_format = "%Y-%m-%d"

In [2]:
# Some helper functions
def date_to_int(date_string, form=_default_date_format):
    """Return date date_string in format form as an integer"""
    return dt.datetime.strptime(date_string, form).toordinal()

def int_to_date(ordinal, form=_default_date_format):
    """Return the day number ordinal to as a string, formatted with form"""
    return dt.datetime.fromordinal(ordinal).strftime(form)

In [3]:
%%time
# Read in data and adjust a few features
# This takes a little time to run (~20 seconds on my desktop)
hospitalizations = pd.read_csv("COVID-19_Reported_Patient_Impact"\
                    "_and_Hospital_Capacity_by_State_Timeseries.csv").copy()
cases = pd.read_csv("United_States_COVID-19_Cases"\
                    "_and_Deaths_by_State_over_Time.csv").copy()
vaccinations = pd.read_csv("COVID-19_Vaccinations"\
                           "_in_the_United_States_Jurisdiction.csv").copy()

cases.rename({"submission_date": "date"}, axis=1, inplace=True)
vaccinations.rename({"Date": "date",
                    "Location": "state"}, axis=1, inplace=True)

hospitalizations["day_number"] = hospitalizations.date.copy()
cases["day_number"] = cases.date.copy()
vaccinations["day_number"] = vaccinations.date.copy()
for n in hospitalizations.index:
    hospitalizations.loc[n, "day_number"] = date_to_int(
                                hospitalizations.loc[n,"date"], "%Y/%m/%d")
for n in cases.index:
    cases.loc[n, "day_number"] = date_to_int(
                                cases.loc[n, "date"], "%m/%d/%Y")
for n in vaccinations.index:
    vaccinations.loc[n, "day_number"] = date_to_int(
                                vaccinations.loc[n, "date"], "%m/%d/%Y")

print("cases date range:           ",
      int_to_date(cases.day_number.min()),
      "to",
      int_to_date(cases.day_number.max()))
print("hospitalizations date range:",
      int_to_date(hospitalizations.day_number.min()),
      "to",
      int_to_date(hospitalizations.day_number.max()))
print("vaccinations date range:    ",
      int_to_date(vaccinations.day_number.min()),
      "to",
      int_to_date(vaccinations.day_number.max()))

# Data is spotty until then
start_date = date_to_int("2020-03-28")
end_date = cases.day_number.max()

print()
print("Using dates",
     int_to_date(start_date),
     "to",
     int_to_date(end_date))

date_range = range(start_date, end_date+1)

state_list = list(hospitalizations.state.unique())
state_list.remove("VI") # Incomplete
state_list.remove("AS") # Incomplete

# 50 states plus DC and Puerto Rico
assert len(state_list) == 52 

cases date range:            2020-01-22 to 2021-10-17
hospitalizations date range: 2020-01-01 to 2021-10-19
vaccinations date range:     2020-12-13 to 2021-10-19

Using dates 2020-03-28 to 2021-10-17
CPU times: user 21.2 s, sys: 15.9 ms, total: 21.2 s
Wall time: 21.2 s


In [4]:
%%time
# Vaccination data start on 2020-12-13, but there's no real data until
# the next day. Fill in missing values with 0.
# For some reason this is pretty slow (~1 min), probably could be optimized
first_vaccine = vaccinations.day_number.min() + 1
series = pd.Series(0, index=vaccinations.columns).copy()
for date in date_range:
    if date >= first_vaccine:
        break
    series.date = int_to_date(date)
    series.day_number = date
    for state in state_list:
        series.state = state
        vaccinations = vaccinations.append(series.copy(), ignore_index=True)

CPU times: user 1min 8s, sys: 16.3 ms, total: 1min 8s
Wall time: 1min 8s


In [5]:
# Trim date ranges and states
hospitalizations = hospitalizations.loc[(hospitalizations.day_number >= start_date) \
                                & (hospitalizations.day_number <= end_date)].copy()
cases = cases.loc[(cases.day_number >= start_date) &\
                  (cases.day_number <= end_date)].copy()
vaccinations = vaccinations.loc[(vaccinations.day_number >= start_date) &\
                                (vaccinations.day_number <= end_date)].copy()
hospitalizations = hospitalizations.loc[[(state in state_list)\
                                 for state in hospitalizations.state]].copy()
cases = cases.loc[[(state in state_list) for state in cases.state]].copy()
vaccinations = vaccinations.loc[[(state in state_list)\
                                 for state in vaccinations.state]].copy()

assert len(cases) == len(hospitalizations) == len(vaccinations) \
                  == len(date_range)*len(state_list)

# Sort data frames by date and location, reindex starting from 0
hospitalizations = hospitalizations.sort_values(["day_number","state"]).reset_index(drop=True).copy()
cases = cases.sort_values(["day_number","state"]).reset_index(drop=True).copy()
vaccinations = vaccinations.sort_values(["day_number","state"]).reset_index(drop=True).copy()

# The dates and locations among the three data frames should now be aligned
assert np.all(hospitalizations.state == cases.state)
assert np.all(hospitalizations.state == vaccinations.state)
assert np.all(hospitalizations.day_number == cases.day_number)
assert np.all(hospitalizations.day_number == vaccinations.day_number)

# Create a new data frame with combined data
covid = hospitalizations.join(cases.drop(["date", "state", "day_number"], axis=1)).copy()
covid = covid.join(vaccinations.drop(["date", "state", "day_number"], axis=1)).copy()

covid.date = [int_to_date(date) for date in covid.day_number]

# Show these three columns first
covid = covid[["day_number", "date", "state"] \
              + list(covid.columns.drop(["day_number", "date", "state"]))]

# Some columns have no data
covid.drop(["consent_cases",
            "consent_deaths",
            "conf_death",
            "prob_death",
            "conf_cases",
            "prob_cases",
            "geocoded_state"], axis=1, inplace=True)

# Separate data by state
covid_state = {}
for state in state_list:
    covid_state[state] = covid.loc[covid.state == state].reset_index(drop=True).copy()

In [6]:
# Write data to .csv files
covid.to_csv("covid_data.csv", index=False)

if not isdir("states"):
    mkdir("states")
for state in state_list:
    covid_state[state].to_csv(join("states", f"{state}.csv"), index=False)